In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, pipeline
from datasets import load_dataset
import torch

c:\Users\aleks\Documents\Moje dokumenty\Studia\Semestr 9\USD\USD-trl\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

In [3]:
def convert_labels(example):
    example["label"] = 1 if example["label"] > 2 else 0
    return example

In [4]:

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", model_max_length=256)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)
device = 0 if torch.cuda.is_available() else "cpu"

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
ds = load_dataset("yelp_review_full", split="train")
ds = ds.filter(lambda x: x["label"] != 2, batched=False)
ds = ds.filter(lambda x: len(x["text"]) > 50, batched=False)
ds = ds.filter(lambda x: len(x["text"]) < 150, batched=False)
ds = ds.map(convert_labels)

val = load_dataset("yelp_review_full", split="test")
val = val.filter(lambda x: x["label"] != 2, batched=False)
val = val.filter(lambda x: len(x["text"]) > 50, batched=False)
val = val.filter(lambda x: len(x["text"]) < 150, batched=False)
val = val.map(convert_labels)

Map: 100%|██████████| 3366/3366 [00:00<00:00, 11035.30 examples/s]


In [6]:
def tokenization(example):
    return tokenizer(example["text"], truncation=True, max_length=256)

train_tokenized = ds.map(tokenization, batched=True)
val_tokenized = val.map(tokenization, batched=True)
train_tokenized.set_format(type="torch", columns=['label', 'text', 'input_ids', 'attention_mask'])
val_tokenized.set_format(type="torch", columns=['label', 'text', 'input_ids', 'attention_mask'])

Map:   0%|          | 0/43983 [00:00<?, ? examples/s]

Map: 100%|██████████| 3366/3366 [00:00<00:00, 20905.47 examples/s]


In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="../results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_dir="../results/logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jungliana (piksle). Use `wandb login --relogin` to force relogin


  4%|▎         | 502/13745 [00:43<18:35, 11.87it/s]

{'loss': 0.2295, 'learning_rate': 4.81811567842852e-05, 'epoch': 0.18}


  7%|▋         | 1002/13745 [01:24<17:28, 12.15it/s]

{'loss': 0.1938, 'learning_rate': 4.6362313568570395e-05, 'epoch': 0.36}


...

                                                     
100%|██████████| 13745/13745 [20:01<00:00, 12.55it/s]

{'eval_loss': 0.29075872898101807, 'eval_runtime': 4.1983, 'eval_samples_per_second': 801.75, 'eval_steps_per_second': 50.258, 'epoch': 5.0}


100%|██████████| 13745/13745 [20:16<00:00, 11.30it/s]

{'train_runtime': 1219.7128, 'train_samples_per_second': 180.301, 'train_steps_per_second': 11.269, 'train_loss': 0.07007470710271486, 'epoch': 5.0}


TrainOutput(global_step=13745, training_loss=0.07007470710271486, metrics={'train_runtime': 1219.7128, 'train_samples_per_second': 180.301, 'train_steps_per_second': 11.269, 'train_loss': 0.07007470710271486, 'epoch': 5.0})

In [8]:
# Save the fine-tuned model
model.save_pretrained("../models/yelpBERT")

In [9]:
results = trainer.evaluate()

# Print evaluation results
print(results)

  0%|          | 0/211 [00:00<?, ?it/s]

100%|██████████| 211/211 [00:04<00:00, 48.54it/s]

{'eval_loss': 0.29075872898101807, 'eval_runtime': 4.3713, 'eval_samples_per_second': 770.018, 'eval_steps_per_second': 48.269, 'epoch': 5.0}


In [10]:
text = "Nice atmosphere, I will go there for my birthday."
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
classifier(text)


[{'label': 'POSITIVE', 'score': 0.9997957348823547}]

In [11]:
text = "Not really a good place to spend your evening"
classifier(text)

[{'label': 'NEGATIVE', 'score': 0.9999626874923706}]

In [12]:
text = "I ate a hard roll today"
classifier(text)

[{'label': 'NEGATIVE', 'score': 0.9995110034942627}]